In [0]:
import requests
import pandas as pd
import time
 
def get_pokemon_details(poke_id):
    url = f"https://pokeapi.co/api/v2/pokemon/{poke_id}"
    response = requests.get(url)
    if response.status_code != 200:
        return None
 
    data = response.json()
    stats_dict = {s["stat"]["name"]: s["base_stat"] for s in data.get("stats", [])}
 
    return {
        "id": data.get("id"),
        "name": data.get("name"),
        "base_experience": data.get("base_experience"),
        "height": data.get("height"),
        "is_default": data.get("is_default"),
        "order": data.get("order"),
        "weight": data.get("weight"),
        "abilities": [ab["ability"]["name"] for ab in data.get("abilities", [])],
        "forms": [f["name"] for f in data.get("forms", [])],
        "game_indices": [gi["version"]["name"] for gi in data.get("game_indices", [])],
        "held_items": [hi["item"]["name"] for hi in data.get("held_items", [])],
        "location_area_encounters": data.get("location_area_encounters"),
        "moves": [m["move"]["name"] for m in data.get("moves", [])],
        "sprites": data.get("sprites", {}).get("front_default"),
        "species": data.get("species", {}).get("name"),
        "types": [t["type"]["name"] for t in data.get("types", [])],
        **stats_dict
    }
 
# Descobre o número total de Pokémons disponíveis
def get_total_pokemon():
    url = "https://pokeapi.co/api/v2/pokemon?limit=1"
    response = requests.get(url)
    return response.json()["count"]
 
# Pega todos os Pokémons
total = get_total_pokemon()
pokemon_list = []
 
for i in range(1, total + 1):
    print(f"Buscando Pokémon {i}/{total}")
    pokemon = get_pokemon_details(i)
    if pokemon:
        pokemon_list.append(pokemon)
    time.sleep(0.2)  # Respeita rate limit da API
 
# Cria DataFrame e exporta para CSV
df = pd.DataFrame(pokemon_list)
df.dispay

**CONFIGURAÇÃO DE DIRETORIOS ESTRUTURA MEDELHÃO**

In [0]:
import requests
import pandas as pd
import json
from datetime import datetime
import os

# Configuração de diretórios 

bronze_path = "/Volumes/workspace/default/bronze/"
silver_path = "/Volumes/workspace/default/silver/"
gold_path = "/Volumes/workspace/default/gold/"

# Criar diretórios se não existirem
os.makedirs(bronze_path, exist_ok=True)
os.makedirs(silver_path, exist_ok=True)
os.makedirs(gold_path, exist_ok=True)

# Coordenadas de São Paulo
LATITUDE = -23.55
LONGITUDE = -46.63

print("Diretórios configurados:")
print(f"Bronze: {bronze_path}")
print(f"Silver: {silver_path}")
print(f"Gold: {gold_path}")

**EXTRAIR DADOS DA API DO TEMPO E SALVAR NA CAMADA BRONZE**

In [0]:
def extract_weather_data():
    """Extrai dados meteorológicos da Open-Meteo API"""
    try:
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            'latitude': LATITUDE,
            'longitude': LONGITUDE,
            'hourly': 'temperature_2m,relative_humidity_2m,pressure_msl,precipitation',
            'timezone': 'America/Sao_Paulo',
            'forecast_days': 1
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        # Salvar dados brutos na camada bronze
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{bronze_path}/weather_data_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(filename, json.dumps(data), overwrite=True)
            
        print(f"Dados meteorológicos salvos em: {filename}")
        return data
        
    except Exception as e:
        print(f"Erro na extração de dados meteorológicos: {str(e)}")
        return None

def extract_air_quality_data():
    """Extrai dados de qualidade do ar da OpenAQ API"""
    try:
        url = "https://api.openaq.org/v2/latest"
        params = {
            'coordinates': f"{LATITUDE},{LONGITUDE}",
            'radius': 10000,
            'limit': 100
        }
        
        headers = {
            'accept': 'application/json'
        }
        
        response = requests.get(url, params=params, headers=headers)
        response.raise_for_status()
        
        data = response.json()
        
        # Salvar dados brutos na camada bronze
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{bronze_path}/air_quality_data_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(filename, json.dumps(data), overwrite=True)
            
        print(f"Dados de qualidade do ar salvos em: {filename}")
        return data
        
    except Exception as e:
        print(f"Erro na extração de dados de qualidade do ar: {str(e)}")
        return None


**TRANSFORMAÇÕES TRATAMENTO DOS DADOS E REMOÇÃO DE NULOS**

In [0]:
def transform_weather_data(weather_data):
    """Transforma dados meteorológicos usando Pandas"""
    if not weather_data:
        return None
    
    try:
        # Extrair dados horários
        hourly_data = weather_data.get('hourly', {})
        
        # Criar DataFrame Pandas
        df = pd.DataFrame({
            'timestamp': hourly_data.get('time', []),
            'temperature_2m': hourly_data.get('temperature_2m', []),
            'relative_humidity_2m': hourly_data.get('relative_humidity_2m', []),
            'pressure_msl': hourly_data.get('pressure_msl', []),
            'precipitation': hourly_data.get('precipitation', [])
        })
        
        # Aplicar transformações
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['date'] = df['timestamp'].dt.date
        df['temperature_2m'] = pd.to_numeric(df['temperature_2m'], errors='coerce')
        df['relative_humidity_2m'] = pd.to_numeric(df['relative_humidity_2m'], errors='coerce')
        df['pressure_msl'] = pd.to_numeric(df['pressure_msl'], errors='coerce')
        df['precipitation'] = pd.to_numeric(df['precipitation'], errors='coerce')
        df['extraction_timestamp'] = datetime.now()
        df['latitude'] = LATITUDE
        df['longitude'] = LONGITUDE
        df['location'] = "São Paulo"
        
        # Remover valores nulos
        df = df.dropna(subset=['temperature_2m', 'timestamp'])
        
        return df
        
    except Exception as e:
        print(f"Erro na transformação de dados meteorológicos: {str(e)}")
        return None

def transform_air_quality_data(air_quality_data):
    """Transforma dados de qualidade do ar usando Pandas"""
    if not air_quality_data:
        return None
    
    try:
        # Extrair resultados
        results = air_quality_data.get('results', [])
        
        if not results:
            return None
        
        # Processar cada localização
        processed_data = []
        for result in results:
            location = result.get('location', '')
            measurements = result.get('measurements', [])
            
            for measurement in measurements:
                processed_data.append({
                    'location': location,
                    'parameter': measurement.get('parameter', ''),
                    'value': measurement.get('value', 0),
                    'unit': measurement.get('unit', ''),
                    'last_updated': measurement.get('lastUpdated', ''),
                    'latitude': LATITUDE,
                    'longitude': LONGITUDE
                })
        
        # Criar DataFrame
        df = pd.DataFrame(processed_data)
        
        # Aplicar transformações
        df['last_updated'] = pd.to_datetime(df['last_updated'])
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        df['extraction_timestamp'] = datetime.now()
        df['date'] = datetime.now().date()
        df['parameter'] = df['parameter'].str.lower()
        
        return df
        
    except Exception as e:
        print(f"Erro na transformação de dados de qualidade do ar: {str(e)}")
        return None

**QUALIDADE DOS DADOS - METRICAS DE QUALIDADE**

In [0]:
def generate_data_quality_report(df, source_name):
    """Gera relatório de qualidade de dados"""
    try:
        # Calcular métricas de qualidade
        total_records = len(df)
        null_counts = {}
        
        for column in df.columns:
            null_counts[column] = df[column].isnull().sum()
        
        # Criar relatório
        report = {
            "source": source_name,
            "timestamp": datetime.now().isoformat(),
            "total_records": total_records,
            "null_counts": null_counts,
            "columns": list(df.columns),
            "data_types": {col: str(dtype) for col, dtype in df.dtypes.items()}
        }
        
        # Salvar relatório
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_filename = f"{bronze_path}/quality_report_{source_name}_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(report_filename, json.dumps(report, indent=2), overwrite=True)
            
        print(f"Relatório de qualidade salvo em: {report_filename}")
        return report
        
    except Exception as e:
        print(f"Erro na geração do relatório de qualidade: {str(e)}")
        return None

**PIPELINE PRINCIPAL ( ETL)**

In [0]:
def run_pipeline():
    """Executa o pipeline completo"""
    print("Iniciando pipeline de dados...")
    
    # Extração
    print("Extraindo dados meteorológicos...")
    weather_data = extract_weather_data()
    
    print("Extraindo dados de qualidade do ar...")
    air_quality_data = extract_air_quality_data()
    
    # Transformação
    print("Transformando dados meteorológicos...")
    weather_df = transform_weather_data(weather_data)
    
    print("Transformando dados de qualidade do ar...")
    air_quality_df = transform_air_quality_data(air_quality_data)
    
    # Qualidade de dados
    if weather_df is not None:
        generate_data_quality_report(weather_df, "weather_data")
        
        # Salvar na camada silver (Parquet)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        silver_weather_file = f"{silver_path}/weather_data_{timestamp}.parquet"
        
        # Salvar usando Spark para garantir compatibilidade com Volumes
        spark_df = spark.createDataFrame(weather_df)
        spark_df.write.mode("overwrite").format("parquet").save(silver_weather_file)
        print(f"Dados meteorológicos salvos na silver: {silver_weather_file}")
    
    if air_quality_df is not None:
        generate_data_quality_report(air_quality_df, "air_quality_data")
        
        # Salvar na camada silver (Parquet)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        silver_air_quality_file = f"{silver_path}/air_quality_data_{timestamp}.parquet"
        
        # Salvar usando Spark
        spark_df = spark.createDataFrame(air_quality_df)
        spark_df.write.mode("overwrite").format("parquet").save(silver_air_quality_file)
        print(f"Dados de qualidade do ar salvos na silver: {silver_air_quality_file}")
    
    # Criar dados gold (se ambos os dados estiverem disponíveis)
    if weather_df is not None and air_quality_df is not None:
        create_gold_data(weather_df, air_quality_df)
    
    print("Pipeline concluído com sucesso!")

def create_gold_data(weather_df, air_quality_df):
    """Cria dados finais na camada gold"""
    try:
        # Dados meteorológicos gold
        weather_gold = weather_df[['timestamp', 'date', 'temperature_2m', 
                                 'relative_humidity_2m', 'pressure_msl', 
                                 'precipitation', 'location', 'latitude', 'longitude']].copy()
        
        # Dados qualidade do ar gold (agregação)
        air_quality_gold = air_quality_df.groupby(['parameter', 'date']).agg({
            'last_updated': 'max',
            'value': 'mean',
            'unit': 'first'
        }).reset_index()
        air_quality_gold.rename(columns={'parameter': 'pollutant'}, inplace=True)
        
        # Salvar dados gold
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        gold_weather_file = f"{gold_path}/weather_gold_{timestamp}.parquet"
        gold_air_quality_file = f"{gold_path}/air_quality_gold_{timestamp}.parquet"
        
        # Salvar usando Spark
        spark_weather = spark.createDataFrame(weather_gold)
        spark_air_quality = spark.createDataFrame(air_quality_gold)
        
        spark_weather.write.mode("overwrite").format("parquet").save(gold_weather_file)
        spark_air_quality.write.mode("overwrite").format("parquet").save(gold_air_quality_file)
        
        print("Dados gold criados com sucesso!")
        print(f"Weather Gold: {gold_weather_file}")
        print(f"Air Quality Gold: {gold_air_quality_file}")
        
    except Exception as e:
        print(f"Erro na criação dos dados gold: {str(e)}")


**EXECUTAR PIPELINE**

In [0]:
# Executar pipeline
run_pipeline()

**CHECKLIST PARA VERIFICAR SE OS ARQUIVOS TRATADOS FORAM SALVOS CORRETAMENTE**

In [0]:
def list_files(directory):
    """Lista arquivos no diretório"""
    try:
        files = dbutils.fs.ls(directory)
        print(f"Arquivos em {directory}:")
        for file in files:
            print(f"  - {file.name}")
    except Exception as e:
        print(f"Erro ao listar {directory}: {str(e)}")

print("Verificando arquivos criados...")
list_files(bronze_path)
list_files(silver_path)
list_files(gold_path)


In [0]:
try:
    gold_files = dbutils.fs.ls(gold_path)
    if gold_files:
        sample_file = gold_files[0].path
        print(f"Amostra do arquivo: {sample_file}")
        
        if sample_file.endswith('.parquet'):
            df_sample = spark.read.parquet(sample_file)
            display(df_sample.limit(5))
    else:
        print("Execute o pipeline primeiro para gerar os dados gold")
except Exception as e:
    print(f" {str(e)}")

In [0]:
def extract_weather_data():
    """Extrai dados meteorológicos da Open-Meteo API"""
    try:
        print("Extraindo dados meteorológicos")
        url = "https://api.open-meteo.com/v1/forecast"
        params = {
            'latitude': LATITUDE,
            'longitude': LONGITUDE,
            'hourly': 'temperature_2m,relative_humidity_2m,pressure_msl,precipitation',
            'timezone': 'America/Sao_Paulo',
            'forecast_days': 1
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        # Salvar dados brutos na camada bronze
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{bronze_path}/weather_data_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(filename, json.dumps(data), overwrite=True)
            
        print(f"Dados meteorológicos salvos em: {filename}")
        
        # Visualizar estrutura dos dados brutos
        print("Estrutura dos dados meteorológicos brutos:")
        print(f"Chaves disponíveis: {list(data.keys())}")
        if 'hourly' in data:
            print(f"Colunas hourly: {list(data['hourly'].keys())}")
            print(f"Número de registros: {len(data['hourly'].get('time', []))}")
        
        return data
        
    except Exception as e:
        print(f"Erro na extração de dados meteorológicos: {str(e)}")
        return None

def extract_air_quality_data():
    """Extrai dados de qualidade do ar usando API alternativa"""
    try:
        print("Extraindo dados de qualidade do ar...")
        
        # Usando API de qualidade do ar do Open-Meteo
        url = "https://air-quality-api.open-meteo.com/v1/air-quality"
        params = {
            'latitude': LATITUDE,
            'longitude': LONGITUDE,
            'hourly': 'pm10,pm2_5,carbon_monoxide,nitrogen_dioxide',
            'timezone': 'America/Sao_Paulo',
            'forecast_days': 1
        }
        
        response = requests.get(url, params=params)
        response.raise_for_status()
        
        data = response.json()
        
        # Salvar dados brutos na camada bronze
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{bronze_path}/air_quality_data_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(filename, json.dumps(data), overwrite=True)
            
        print(f"ados de qualidade do ar salvos em: {filename}")
        
        # Visualizar estrutura dos dados brutos
        print("Estrutura dos dados de qualidade do ar brutos:")
        print(f"Chaves disponíveis: {list(data.keys())}")
        if 'hourly' in data:
            print(f"Colunas hourly: {list(data['hourly'].keys())}")
            print(f"Número de registros: {len(data['hourly'].get('time', []))}")
        
        return data
        
    except Exception as e:
        print(f"Erro na extração de dados de qualidade do ar: {str(e)}")
        print("Criando dados de exemplo para demonstração...")
        
        # Criar dados de exemplo para continuar o pipeline
        example_data = {
            "hourly": {
                "time": [(datetime.now() - pd.Timedelta(hours=i)).isoformat() for i in range(24)],
                "pm10": [25.5 + i for i in range(24)],
                "pm2_5": [15.2 + i*0.5 for i in range(24)],
                "carbon_monoxide": [0.8 + i*0.1 for i in range(24)],
                "nitrogen_dioxide": [12.3 + i*0.8 for i in range(24)]
            }
        }
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{bronze_path}/air_quality_data_example_{timestamp}.json"
        dbutils.fs.put(filename, json.dumps(example_data), overwrite=True)
        
        print(f"Dados de exemplo salvos em: {filename}")
        return example_data


**TRATAMENTO V2**

In [0]:
def transform_weather_data(weather_data):
    """Transforma dados meteorológicos usando Pandas"""
    if not weather_data:
        return None
    
    try:
        print("Transformando dados meteorológicos...")
        # Extrair dados horários
        hourly_data = weather_data.get('hourly', {})
        
        # Criar DataFrame Pandas
        df = pd.DataFrame({
            'timestamp': hourly_data.get('time', []),
            'temperature_2m': hourly_data.get('temperature_2m', []),
            'relative_humidity_2m': hourly_data.get('relative_humidity_2m', []),
            'pressure_msl': hourly_data.get('pressure_msl', []),
            'precipitation': hourly_data.get('precipitation', [])
        })
        
        # Aplicar transformações
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['date'] = df['timestamp'].dt.date
        df['temperature_2m'] = pd.to_numeric(df['temperature_2m'], errors='coerce')
        df['relative_humidity_2m'] = pd.to_numeric(df['relative_humidity_2m'], errors='coerce')
        df['pressure_msl'] = pd.to_numeric(df['pressure_msl'], errors='coerce')
        df['precipitation'] = pd.to_numeric(df['precipitation'], errors='coerce')
        df['extraction_timestamp'] = datetime.now()
        df['latitude'] = LATITUDE
        df['longitude'] = LONGITUDE
        df['location'] = "São Paulo"
        
        # Remover valores nulos
        initial_count = len(df)
        df = df.dropna(subset=['temperature_2m', 'timestamp'])
        final_count = len(df)
        
        print(f"Dados meteorológicos transformados:")
        print(f"Registros iniciais: {initial_count}")
        print(f"Registros após limpeza: {final_count}")
        print(f"Colunas: {list(df.columns)}")
        print(f"Período: {df['timestamp'].min()} até {df['timestamp'].max()}")
        
        # Visualizar amostra dos dados transformados
        print(" Amostra dos dados meteorológicos transformados:")
        display(df.head(5))
        
        return df
        
    except Exception as e:
        print(f"Erro na transformação de dados meteorológicos: {str(e)}")
        return None

def transform_air_quality_data(air_quality_data):
    """Transforma dados de qualidade do ar usando Pandas"""
    if not air_quality_data:
        return None
    
    try:
        print("Transformando dados de qualidade do ar...")
        # Extrair dados horários
        hourly_data = air_quality_data.get('hourly', {})
        
        # Criar DataFrame
        df = pd.DataFrame({
            'timestamp': hourly_data.get('time', []),
            'pm10': hourly_data.get('pm10', []),
            'pm2_5': hourly_data.get('pm2_5', []),
            'carbon_monoxide': hourly_data.get('carbon_monoxide', []),
            'nitrogen_dioxide': hourly_data.get('nitrogen_dioxide', [])
        })
        
        # Aplicar transformações
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df['date'] = df['timestamp'].dt.date
        df['pm10'] = pd.to_numeric(df['pm10'], errors='coerce')
        df['pm2_5'] = pd.to_numeric(df['pm2_5'], errors='coerce')
        df['carbon_monoxide'] = pd.to_numeric(df['carbon_monoxide'], errors='coerce')
        df['nitrogen_dioxide'] = pd.to_numeric(df['nitrogen_dioxide'], errors='coerce')
        df['extraction_timestamp'] = datetime.now()
        df['latitude'] = LATITUDE
        df['longitude'] = LONGITUDE
        df['location'] = "São Paulo"
        
        # Remover valores nulos
        initial_count = len(df)
        df = df.dropna(subset=['pm10', 'timestamp'])
        final_count = len(df)
        
        print(f"Dados de qualidade do ar transformados:")
        print(f"Registros iniciais: {initial_count}")
        print(f"Registros após limpeza: {final_count}")
        print(f"Colunas: {list(df.columns)}")
        print(f"Período: {df['timestamp'].min()} até {df['timestamp'].max()}")
        
        # Visualizar amostra dos dados transformados
        print("Amostra dos dados de qualidade do ar transformados:")
        display(df.head(5))
        
        return df
        
    except Exception as e:
        print(f"Erro na transformação de dados de qualidade do ar: {str(e)}")
        return None


In [0]:
def generate_data_quality_report(df, source_name):
    """Gera relatório de qualidade de dados"""
    try:
        print(f"Gerando relatório de qualidade para {source_name}...")
        # Calcular métricas de qualidade
        total_records = len(df)
        null_counts = {}
        
        for column in df.columns:
            null_counts[column] = int(df[column].isnull().sum())  # Converter para int nativo
        
        # Estatísticas básicas para colunas numéricas
        numeric_stats = {}
        numeric_columns = df.select_dtypes(include=['number']).columns
        for col in numeric_columns:
            numeric_stats[col] = {
                'min': float(df[col].min()),
                'max': float(df[col].max()),
                'mean': float(df[col].mean()),
                'std': float(df[col].std())
            }
        
        # Converter tipos de dados para serialização JSON
        data_types = {}
        for col, dtype in df.dtypes.items():
            data_types[col] = str(dtype)
        
        # Criar relatório
        report = {
            "source": source_name,
            "timestamp": datetime.now().isoformat(),
            "total_records": int(total_records),
            "null_counts": null_counts,
            "numeric_statistics": numeric_stats,
            "columns": list(df.columns),
            "data_types": data_types
        }
        
        # Salvar relatório
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        report_filename = f"{bronze_path}/quality_report_{source_name}_{timestamp}.json"
        
        # Usar dbutils para salvar no Volume
        dbutils.fs.put(report_filename, json.dumps(report, indent=2), overwrite=True)
            
        print(f"Relatório de qualidade salvo em: {report_filename}")
        
        # Visualizar resumo do relatório
        print("Resumo do Relatório de Qualidade:")
        print(f"Total de registros: {total_records}")
        print(f"Colunas com valores nulos:")
        for col, null_count in null_counts.items():
            if null_count > 0:
                print(f"     - {col}: {null_count} nulos")
        
        return report
        
    except Exception as e:
        print(f"Erro na geração do relatório de qualidade: {str(e)}")
        return None

In [0]:
def run_pipeline():
    """Executa o pipeline completo"""
    print("Iniciando pipeline de dados...")
    print("=" * 50)
    
    # Extração
    print("\nETAPA 1: EXTRAÇÃO")
    print("-" * 30)
    weather_data = extract_weather_data()
    air_quality_data = extract_air_quality_data()
    
    # Transformação
    print("\nETAPA 2: TRANSFORMAÇÃO")
    print("-" * 30)
    weather_df = transform_weather_data(weather_data)
    air_quality_df = transform_air_quality_data(air_quality_data)
    
    # Qualidade de dados
    print("\n ETAPA 3: QUALIDADE DE DADOS")
    print("-" * 30)
    if weather_df is not None:
        weather_report = generate_data_quality_report(weather_df, "weather_data")
    
    if air_quality_df is not None:
        air_quality_report = generate_data_quality_report(air_quality_df, "air_quality_data")
    
    # Salvamento Silver
    print("\n ETAPA 4: CAMADA SILVER")
    print("-" * 30)
    if weather_df is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        silver_weather_file = f"{silver_path}/weather_data_{timestamp}.parquet"
        
        # Salvar usando Spark
        spark_df = spark.createDataFrame(weather_df)
        spark_df.write.mode("overwrite").format("parquet").save(silver_weather_file)
        print(f" Dados meteorológicos salvos na silver: {silver_weather_file}")
        
        # Visualizar dados silver
        print(" Dados salvos na Silver (Weather):")
        display(spark_df.limit(5))
    
    if air_quality_df is not None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        silver_air_quality_file = f"{silver_path}/air_quality_data_{timestamp}.parquet"
        
        # Salvar usando Spark
        spark_df = spark.createDataFrame(air_quality_df)
        spark_df.write.mode("overwrite").format("parquet").save(silver_air_quality_file)
        print(f" Dados de qualidade do ar salvos na silver: {silver_air_quality_file}")
        
        # Visualizar dados silver
        print(" Dados salvos na Silver (Air Quality):")
        display(spark_df.limit(5))
    
    # Criar dados gold
    print("\n ETAPA 5: CAMADA GOLD")
    print("-" * 30)
    if weather_df is not None and air_quality_df is not None:
        create_gold_data(weather_df, air_quality_df)
    
    print("\n" + "=" * 50)
    print("PIPELINE CONCLUÍDO COM SUCESSO!")
    print("=" * 50)

def create_gold_data(weather_df, air_quality_df):
    """Cria dados finais na camada gold"""
    try:
        print(" Criando camada gold...")
        
        # Dados meteorológicos gold
        weather_gold = weather_df[['timestamp', 'date', 'temperature_2m', 
                                 'relative_humidity_2m', 'pressure_msl', 
                                 'precipitation', 'location', 'latitude', 'longitude']].copy()
        
        # Dados qualidade do ar gold
        air_quality_gold = air_quality_df[['timestamp', 'date', 'pm10', 'pm2_5', 
                                         'carbon_monoxide', 'nitrogen_dioxide',
                                         'location', 'latitude', 'longitude']].copy()
        
        # Salvar dados gold
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        gold_weather_file = f"{gold_path}/weather_gold_{timestamp}.parquet"
        gold_air_quality_file = f"{gold_path}/air_quality_gold_{timestamp}.parquet"
        
        # Salvar usando Spark
        spark_weather = spark.createDataFrame(weather_gold)
        spark_air_quality = spark.createDataFrame(air_quality_gold)
        
        spark_weather.write.mode("overwrite").format("parquet").save(gold_weather_file)
        spark_air_quality.write.mode("overwrite").format("parquet").save(gold_air_quality_file)
        
        print("Dados gold criados com sucesso!")
        print(f"  Weather Gold: {gold_weather_file}")
        print(f"  Air Quality Gold: {gold_air_quality_file}")
        
        # Visualização completa dos dados gold
        print("\n DADOS GOLD - VISUALIZAÇÃO COMPLETA")
        print(" DADOS METEOROLÓGICOS GOLD:")
        display(spark_weather.limit(10))
        
        print(" DADOS DE QUALIDADE DO AR GOLD:")
        display(spark_air_quality.limit(10))
        
        # Estatísticas resumidas
        print(" ESTATÍSTICAS RESUMIDAS:")
        print("Temperatura média:", spark_weather.select(avg('temperature_2m')).collect()[0][0])
        print("PM2.5 médio:", spark_air_quality.select(avg('pm2_5')).collect()[0][0])
        
    except Exception as e:
        print(f" Erro na criação dos dados gold: {str(e)}")


In [0]:
# Executar pipeline
run_pipeline()

In [0]:
# Verificar arquivos criados
def list_files(directory):
    """Lista arquivos no diretório"""
    try:
        files = dbutils.fs.ls(directory)
        print(f" Arquivos em {directory}:")
        for file in files:
            print(f"  - {file.name} ({file.size} bytes)")
        return len(files)
    except Exception as e:
        print(f" Erro ao listar {directory}: {str(e)}")
        return 0

print(" VERIFICAÇÃO FINAL DOS ARQUIVOS CRIADOS")
print("=" * 50)

print("\n CAMADA BRONZE:")
bronze_count = list_files(bronze_path)

print("\n CAMADA SILVER:")
silver_count = list_files(silver_path)

print("\n CAMADA GOLD:")
gold_count = list_files(gold_path)

print(f"\n RESUMO FINAL:")
print(f"Bronze: {bronze_count} arquivos")
print(f" Silver: {silver_count} arquivos")
print(f" Gold: {gold_count} arquivos")


In [0]:
try:
    # Ler dados gold para visualização
    gold_weather_files = [f.path for f in dbutils.fs.ls(gold_path) if 'weather_gold' in f.name]
    gold_air_files = [f.path for f in dbutils.fs.ls(gold_path) if 'air_quality_gold' in f.name]
    
    if gold_weather_files and gold_air_files:
        weather_df = spark.read.parquet(gold_weather_files[0])
        air_df = spark.read.parquet(gold_air_files[0])
        
        print(" GRÁFICOS DOS DADOS GOLD")
        print("  Variação da Temperatura:")
        display(weather_df.select('timestamp', 'temperature_2m').orderBy('timestamp').limit(24))
        
        print("  Níveis de PM2.5:")
        display(air_df.select('timestamp', 'pm2_5').orderBy('timestamp').limit(24))
        
except Exception as e:
    print(f"  Visualização adicional não disponível: {str(e)}")